# Suma de numeros binarios

El objetivo de esta práctica es diseñar un modelo recurrente basado en modelos de redes neuronales (RNN). Empecemos por cargar las librerías necesarias. 

In [2]:
import os
import tensorflow as tf
from d2l import tensorflow as d2l

import tensorflow.keras as keras
import numpy as np

La idea es sencilla. Todo número entero tiene una representación binaria. Por ejemplo, 
el número `43` tiene una representación binaria igual a `1101010`. La cual podemos representar como un expansión en potencias de 2. Es decir, 

$${\color{blue}1} \times 2^0 + {\color{blue}1}\times2^1+ {\color{blue}0}\times 2^2 + {\color{blue}1}\times 2^3 + {\color{blue}0}\times 2^4+ {\color{blue}1}\times2^5+ {\color{blue}0}\times2^6$$ 

El objetivo es, dados dos números en binario ($x_1, x_2$) queremos entrenar una RNN para producir el resultado $y = x_1 + x_2.$

Por ejemplo, tenemos los siguientes dos números:


In [94]:
np.random.seed(1)

x1 = np.random.randint(0, 2**(7-1))
x2 = np.random.randint(0, 2**(7-1))

print("[%d, %d]"%(x1, x2))

[37, 43]


Que si sumamos tenemos como resultado:

In [96]:
print("x1 + x2 = %d"%(x1+x2))

x1 + x2 = 80


Lo que queremos hacer es encontrar una función (RNN) que "sepa" sumar en representación binaria. Es decir, utilizando la representación:

In [121]:
format_str = '{:0' + str(sequence_len) + 'b}'

print("x1 = ", ''.join(list(reversed(format_str.format(x1)))))
print("x2 = ", ''.join(list(reversed(format_str.format(x2)))))

x1 =  1010010
x2 =  1101010


In [123]:
print("x1 + x2 = ", ''.join(list(reversed(format_str.format(x1 + x2)))))

x1 + x2 =  0000101


**Nota** que estamos utilizando la convención de escribir un número binario en potencias crecientes de 2. La suma binaria en este caso es una operación que va de izquierda a derecha. Con las reglas usuales: 

$$
\begin{align}
0 + 0 &= 0\,,\\
1 + 0 &= 1\,,\\
0 + 1 &= 1\,.\\
\end{align}
$$

El caso $1 + 1 = 10$ es el mas interesante, pues en un modelo secuencial que predice un digito a la vez tendría que asignar un `0` y "saber" que "lleva" un `1`.

Vamos a escribir el modelo mas sencillo que podamos. Para esto consideramos lo siguiente. La suma binaria la entendemos digito por digito y con esto tratamos de decidir si emitimos un `0` o un `1`. Para esta parte supondremos que sumamos número pequeños. 

**Q1** ¿Cuál es el número más grande que podemos encontrar si generamos numeros binarias de longitud igual a 7?

**Q2** Llena los espacios que faltan en el codigo de abajo para definir un modelo RNN simple.

In [126]:
MAX_BIT = 7


input_dim = # El numero de dimensiones de entrada.
activation = # Una cadena de texto especificando la función de activación en la capa de salida. 

model = keras.models.Sequential()
model.add(keras.layers.SimpleRNN(
    4,
    input_dim        = input_dim,
    return_sequences = True
))
model.add(keras.layers.Dense(2, activation='relu'))
model.add(keras.layers.Dense(1, activation=activation))

print(model.input_shape)
print(model.output_shape)

(None, None, 2)
(None, None, 1)


**Q3** Prueba que el modelo funciona con un ejemplo sencillo. 

In [ ]:
# test if the prediction shape are expected
input_array = #
x = np.array([input_array])
print(x.shape)
model.predict(x)

Para evitar que el codigo sea demasiado complejo, puedes utilizar las siguientes funciones que modifican las entradas para representarlas en binarios. Nota que necesitamos una función adicional para hacer _padding_ y rellenar con ceros la secuencia cuando el número es muy pequeño relativo a la longitud de potencias que estamos utilizando.  

In [25]:
import keras.preprocessing.sequence

def to_seq(i):
    return list(reversed(list(map(float, "{0:b}".format(i)))))

def pad_seq(a, b, c, maxlen=None):
    return keras.preprocessing.sequence.pad_sequences(
        [a, b, c],
        padding='post',
        dtype='float32',
        maxlen=maxlen
    )

Abajo encontrarás dos funciones mas para generar conjuntos de datos para entrenar. 

In [26]:
def gen_sample(a = None, b = None):
    maxlen = None
    if a is None and b is None:
        a = np.random.randint(2 ** MAX_BIT)
        b = np.random.randint(2 ** MAX_BIT)
        maxlen = MAX_BIT + 1
    c = a + b
    a, b, c = pad_seq(to_seq(a), to_seq(b), to_seq(c), maxlen=maxlen)

    return np.array(list(zip(a, b))), c
def gen_mass_samples(n = 50):
    x = np.zeros((n, MAX_BIT + 1, 2))
    y = np.zeros((n, MAX_BIT + 1, 1))
    for i in range(n):
        x_, y_ = gen_sample()
        x[i, :, :] = x_
        y[i, :, :] = y_.reshape(1, -1, 1)
    return x, y

In [27]:
x, y = gen_mass_samples()
print(x.shape)
print(y.shape)
list(zip(x[0], y[0]))

(50, 11, 2)
(50, 11, 1)


[(array([1., 0.]), array([1.])),
 (array([0., 0.]), array([0.])),
 (array([1., 0.]), array([1.])),
 (array([1., 1.]), array([0.])),
 (array([0., 0.]), array([1.])),
 (array([1., 0.]), array([1.])),
 (array([1., 1.]), array([0.])),
 (array([1., 0.]), array([0.])),
 (array([0., 0.]), array([1.])),
 (array([1., 0.]), array([1.])),
 (array([0., 0.]), array([0.]))]

El siguiente pedazo de código entrena el modelo con elecciones _default_. 

In [129]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
N = 10000
for i in range(8):
    x, y = gen_mass_samples(N)
    model.fit(x.reshape(N, -1, 2), y.reshape(N, -1, 1), batch_size=50, epochs=3)

Epoch 1/10
200/200 [==============================] - 1s 1ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 2/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 3/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 4/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 5/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 6/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 7/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 8/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 9/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 10/10
200/200 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 1.0000

200/200 [==============================] - 0s 1ms/step - loss: 7.2942e-06 - accuracy: 1.0000


In [130]:
x, y = gen_mass_samples(N * 3)
model.evaluate(x.reshape(N * 3, -1, 2), y.reshape(N * 3, -1, 1))

938/938 [==============================] - 1s 834us/step - loss: 6.9630e-06 - accuracy: 1.0000


[6.962984116398729e-06, 1.0]

**Q4** Escribe un ejemplo para verificar la capacidad predictiva del modelo. 

**Q5** Copia y pega el código necesario y entrena el modelo con números mas grandes. Evalúa la capacidad predictiva del modelo en este escenario (números grandes, por ejemplo longitud en binario = 30). ¿Porqué crees que ocurra esto?

**Q6** Extiende el modelo para incorporar celdas de memoria y evalúa si el fenomeno que observaste en la pregunta anterior sigue persistiendo (posiblemente tengas que incrementar el número de epocas).